# Upload data and visualize
I need to grab the beta value and the standard error (beta) for each SNP accross each cohort.

**Don't forget** I need to grab the minor allele and major allele of each SNP across each cohort. I will determine the orientation. Then, I will use a "majority rules" approach to determine if I need to flip the sign of the beta. Specifically, if the major and minor alleles are listed differently for a few cohorts, then I will need to flip the sign of the beta (i.e. change from negative to positive or vice versa.)

**Also** run test of code against `rs16969968`

## Sample data
This sample data is for one variant - `rs1008078` - accross all the cohorts in meta-analysis 044. I will store the results in a dictionary. The key will be the rsID and the value will be a pandas dataframe.

In [2]:
os.listdir()

['.sh',
 '044_combine_data_mrBig.tsv',
 '044_meta_analysis_mrBig_example.xlsx',
 'AAND_COGEND2_AA.sh',
 'AAND_COGEND2_AA.txt',
 'Book1.xlsx',
 'COGEND2_AA.sh',
 'COGEND2_AA.txt',
 'COGEND2_EA.sh',
 'COGEND2_EA.txt',
 'COGEND_AA.sh',
 'COGEND_AA.txt',
 'COGEND_EA.sh',
 'COGEND_EA.txt',
 'COPDGene_AA.sh',
 'COPDGene_AA.txt',
 'COPDGene_EA.sh',
 'COPDGene_EA.txt',
 'deCODE',
 'deCODE_EA.txt',
 'deCODE_EA.txt.bak',
 'Dental_Caries_EA.sh',
 'Dental_Caries_EA.txt',
 'EAGLE_EA.sh',
 'EAGLE_EA.txt',
 'FINN_TWIN_EA.sh',
 'FINN_TWIN_EA.txt',
 'GAIN_AA.sh',
 'GAIN_AA.txt',
 'GAIN_EA.sh',
 'GAIN_EA.txt',
 'JHS_AA.sh',
 'JHS_AA.txt',
 'merge.R',
 'mrBig_sample',
 'mrBig_sample.bak',
 'nonGAIN_EA.sh',
 'nonGAIN_EA.txt',
 'NTR_EA.sh',
 'NTR_EA.txt',
 'SAGE_AA.sh',
 'SAGE_AA.txt',
 'SAGE_EA.sh',
 'SAGE_EA.txt',
 'SNPlist.txt',
 'studies.json',
 'UW_TTURC_AA.sh',
 'UW_TTURC_AA.txt',
 'UW_TTURC_EA.sh',
 'UW_TTURC_EA.txt',
 'YALE_PENN_AA.sh',
 'YALE_PENN_AA.txt',
 'YALE_PENN_EA.sh',
 'YALE_PENN_EA.txt',


In [67]:
import os
import pandas as pd
import numpy as np
import math
from decimal import Decimal
from scipy.stats import chi2

# I will create a new data frame for each variant. This data frame will have the column names:
# (1) cohort, (2) Ancestry group, (3) Beta, (4) Std. Error, (5) Seweighted, and (6) Pr(>|t|) which is the p-val
# addictionally  I will add the rows for calculating the meta in the Seweighted column and below all of the cohorts
# getting the first cohort, note that this will eventually be in a loop of the cohorts

#os.chdir("C:\\Users\\jmarks\\Desktop\\Projects\\Nicotine\\GSCAN_extended_results_nicotine\\results\\results_from_missing_snp_lookup\\")
os.chdir("C:\\Users\\jmarks\\Desktop\\")
mydata = pd.ExcelFile("mrBig_actual_data.xlsx")
mydata = mydata.parse("Sheet1")

cohorts_list44 = ["AAND_AA",
"COGEND_AA",
"COGEND_EA",
"COGEND2_AA",
"COGEND2_EA",
"COPDGENE_AA",
"COPDGENE_EA",
"DECODE_EA",
"DENTAL_CARIES_EA",
"EAGLE_EA",
"FINN_TWIN_EA",
"GAIN_AA",
"GAIN_EA",
"JHS_ARIC_AA",
"NONGAIN_EA",
"NTR_EA",
"SAGE_AA",
"SAGE_EA",
"UW_TTURC_AA",
"UW.TTURC_EA",
"YALE_PENN_AA",
"YALE_PENN_EA"]

totalRows = len(cohorts_list44)

# This dictionary will have an rsID for the key and the key value will be a dataframe
dataDict = {}

# initialize a dataframe
emptyArray = np.empty((totalRows,13,))
emptyArray[:] = np.nan
columns = ["SNP", "Cohort", "Ancestry group", "Beta", "Std. Error", "Seweighted", "Pr(>|t|)", 
           "AllMeta.SumSEweight", "AllMeta.weightedSE", "AllMeta.SEweighted_beta", 
           "AllMeta.SEweighted_Z", "AllMeta.SEweighted_Chi", "AllMeta.SEweighted_P"]
num_of_rsIDs = len(mydata)
#print(num_of_rsIDs)

# Above this write a script which removes the variants who were not present in any of the cohorts




# loop to fill in information for the meta-anlaysis calculation
for rsID in range(num_of_rsIDs):
    
    markerName = mydata.iloc[rsID,0]
    dataDict[markerName] = pd.DataFrame(columns=columns, data=emptyArray)
    dataDict[markerName].iloc[0,0] = markerName # add SNP
    metaSEweighted_beta = 0
    for cohort in range(len(cohorts_list44)):
        
        # get all of the cohort specific data
        cohortData = mydata.filter(like=cohorts_list44[cohort]).iloc[rsID,:]
        # add cohort to dataframe
        cohortName = cohorts_list44[cohort]
        dataDict[markerName].iloc[cohort,1] = cohorts_list44[cohort][0:-3]
            
        # add Ancestry group
        ancestry = cohorts_list44[cohort][-2:]
        dataDict[markerName].iloc[cohort, 2] = ancestry
        # add Beta
        betaVal = cohortData.filter(regex="^beta")[0]
        
        # flip the sign for FINN_Tw
        if cohorts_list44[cohort] == "1DECODE_EA" or cohorts_list44[cohort] == "FINN_TWIN_EA":
            betaVal = -betaVal
        dataDict[markerName].iloc[cohort, 3] = betaVal
        
        # add Std. Error
        standardErr = cohortData.filter(like="sebeta")[0]
        if cohorts_list44[cohort] == "YALE_PENN_EA":
            standardErr = 0.023608174
        dataDict[markerName].iloc[cohort, 4] = standardErr
        
        # add Seweighted
        seWeighted = 1 / (standardErr ** 2)
        dataDict[markerName].iloc[cohort,5]  = seWeighted
        
        # add p-val
        pVal = cohortData.filter(regex="^p.")[0]
        dataDict[markerName].iloc[cohort, 6] = pVal
        
        #  metaSEweighted_beta calculation 
        if not np.isnan(betaVal):
            metaSEweighted_beta += (betaVal*seWeighted)
      
    # Meta calculations
    SumSEweight = dataDict[markerName]['Seweighted'].sum()
    dataDict[markerName].iloc[0, 7] = SumSEweight

    metaWeightedSE = math.sqrt(1/SumSEweight)
    dataDict[markerName].iloc[0, 8] = metaWeightedSE
    
    metaSEweighted_beta = metaSEweighted_beta / SumSEweight 
    dataDict[markerName].iloc[0, 9] = metaSEweighted_beta

    metaSEweighted_Z = (metaSEweighted_beta / metaWeightedSE)
    dataDict[markerName].iloc[0, 10] = metaSEweighted_Z

    metaSEweighted_chi = metaSEweighted_Z ** 2
    dataDict[markerName].iloc[0, 11] = metaSEweighted_chi

    metaSEweighted_P = '%.2E' % Decimal(chi2.sf(metaSEweighted_chi, 1))
    dataDict[markerName].iloc[0, 12] = metaSEweighted_P


#print(dataDict[markerName]["Std. Error"].to_string(index=False))
#print(dataDict[markerName]["Beta"].to_string(index=False))
#print(dataDict[markerName]["Pr(>|t|)"].to_string(index=False))

dataDict[markerName]
#dataDict["rs1022528"]
#dataDict[markerName].to_csv("C:\\Users\\jmarks\\Desktop\\out.file", sep='\t', index=False)itei

,SNP,Cohort,Ancestry group,Beta,Std. Error,Seweighted,Pr(>|t|),AllMeta.SumSEweight,AllMeta.weightedSE,AllMeta.SEweighted_beta,AllMeta.SEweighted_Z,AllMeta.SEweighted_Chi,AllMeta.SEweighted_P
0,rs16969968,AAND,AA,-0.006330,0.049744,404.127673,8.987366e-01,25633.092688,0.006246,-0.063454,-10.159237,103.210095,3.01E-24
1,NaN,COGEND,AA,-0.183989,0.090620,121.772978,4.232268e-02,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,COGEND,EA,-0.123617,0.026794,1392.863909,3.959034e-06,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,COGEND2,AA,-0.021570,0.093901,113.412870,8.183132e-01,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,COGEND2,EA,-0.089653,0.064377,241.286561,1.637336e-01,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,COPDGENE,AA,-0.044872,0.048550,424.256679,3.553569e-01,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,COPDGENE,EA,-0.071561,0.023476,1814.462924,2.301693e-03,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,DECODE,EA,-0.068100,0.011800,7181.844298,8.329000e-09,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,DENTAL_CARIES,EA,-0.082382,0.062245,258.102644,1.856666e-01,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,EAGLE,EA,-0.025725,0.018782,2834.760206,1.708006e-01,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
for key in dataDict:
    print(key)
    print(dataDict[key])
    #dataDict[key]

rs1008078
          SNP            Cohort Ancestry group      Beta  Std. Error  \
0   rs1008078   AAND_COGEND2_AA             AA  0.022789    0.038459   
1         NaN         COGEND_AA             AA -0.041462    0.064375   
2         NaN         COGEND_EA             EA -0.022469    0.026324   
3         NaN        COGEND2_AA             AA  0.022789    0.038459   
4         NaN        COGEND2_EA             EA -0.052799    0.068863   
5         NaN       COPDGene_AA             AA -0.033628    0.035596   
6         NaN       COPDGene_EA             EA  0.005601    0.022238   
7         NaN         deCODE_EA             EA  0.001000    0.011700   
8         NaN  Dental_Caries_EA             EA  0.053374    0.062319   
9         NaN          EAGLE_EA             EA  0.000272    0.019446   
10        NaN      FINN_TWIN_EA             EA  0.003087    0.021316   
11        NaN           GAIN_AA             AA -0.160170    0.076582   
12        NaN           GAIN_EA             EA  0.0266

## Need to determine what to do with NaN values in my data.
### Also, if data is NaN for all cohorts, need to remove them from my data (compile a list of these SNPs)
ask Dana how these go into calculation

In [298]:
cohorts_list44[5]

'COPDGene_AA'

In [150]:
import numpy as np
a = np.empty((29,7,))
a[:] = np.nan
print(a)
tempDF = pd.DataFrame(np.nan, index=[0,1,2,3],columns=["a","b","c"])
#tempDF

[[ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan 

In [112]:
dataDict['rs1008078']

,SNP,Cohort,Ancestry group,Beta,Std. Error,Seweighted,Pr(>|t|)


# 044 meta-analysis

In [51]:
import os
import pandas as pd



os.chdir("C:\\Users\\jmarks\\Desktop\\Projects\\Nicotine\\GSCAN_extended_results_nicotine\\results\\results_from_missing_snp_lookup\\")


#os.listdir()
#xl = pd.ExcelFile("missing_SNPs_results_prefiltered_meta_analyses_044_045_046_V02.xlsx")
xl.sheet_names
zero44 = xl.sheet_names[0]
zero44 = xl.parse(zero44)
zero45 = xl.sheet_names[1]
zero45 = xl.parse(zero45)
zero46 = xl.sheet_names[2]
zero46 = xl.parse(zero46)
zero44.iloc[0:1,:]

# beta is in each, but also sebeta is too.

IndexError: list index out of range

# 045 meta-analysis

# 046 meta-analysis